# Data Retreival

Quake example: 2019-07-16 Earthquake near Byron, California

USGS: https://earthquake.usgs.gov/earthquakes/eventpage/nc73225421/executive

Originally prepared by Andrea Chiang (andrea@llnl.gov)

Fixed and narrated by Alan Yu (alanjyu@outlook.com)

### Objectives

- Learn how to use Obspy
- Retrieve an earthquake catalogue based on our specified criteria.
- Retrieve waveforms based on information on the earthquake catalogue.

To run this tutorial you will need Python 3 and the following packages:

- ObsPy >= 1.0.0
- pandas >= 2.0.0
- matplotlib
- NumPy <= 1.23
- MTtime*

\* To install MTtime, please read README.md. 

First, we install the required modules, mainly Obspy. This is going to help us download the earthquake data we need.

In [7]:
from pathlib import Path
from obspy.clients.fdsn import Client
from obspy import read_events, UTCDateTime
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader

I have set the search variables to download only the earthquake of interest.

In [8]:
client = Client('IRIS')

# search criteria
strtime = UTCDateTime('2019-07-16T00:00:00') # start time
endtime = UTCDateTime('2019-07-16T23:59:59') # end time

# magnitudes, extent, etc.
catalog = client.get_events(starttime = strtime, endtime=endtime,
                            minmagnitude=4, maxmagnitude=5,
                            minlatitude=36, maxlatitude=38,
                            minlongitude=-122, maxlongitude=-120)
catalog.write('catalog.xml', format = 'QUAKEML') # write the catalogue to a localxml file

### Download data
We will download the waveforms and station metadata from the Northern California Earthquake Data Center (NCEDC) using ObsPy's mass_downloader function.

To achieve this, we create a directory for each event, with all downloaded files for that event stored there. In addition to MSEED and STATIONXML files, we will write the event origin information to a text file that will be stored in the current working directory.

In [9]:
# set the time (in seconds) before and after the event origin
time_before = 60
time_after = 300

catalog = read_events('catalog.xml')

# we create a directory for each quake
for event in catalog:
    evid = str(catalog[0].origins[0].resource_id).split('=')[-1] # resource_id as the event id
    outdir = evid
    Path(outdir).mkdir(parents=True, exist_ok=True)
    
    # event origin
    origin_time = event.preferred_origin().time
    strtime = origin_time - time_before
    endtime = origin_time + time_after
    
    # event location
    evlo = event.preferred_origin().longitude
    evla = event.preferred_origin().latitude
    depth = event.preferred_origin().depth # in meters
    
    # search area
    domain = CircularDomain(latitude=evla, longitude=evlo, 
                            minradius=0.7, maxradius=1.3)
    
    # set the search period and additional criteria
    restrictions = Restrictions(starttime=strtime, endtime=endtime,
                                reject_channels_with_gaps=True,
                                minimum_length=0.95,
                                network='BK',
                                channel_priorities=['BH[ZNE12]', 'HH[ZNE12]'],
                                sanitize=True)
    
    # save catalog info to file
    event_out = (
        '{evid:s},{origin:s},{jdate:s},'
        '{lon:.4f},{lat:.4f},{depth:.4f},'
        '{mag:.2f},{auth:s}\n'
        )        

    if event.preferred_magnitude() is None:
        mag = -999.
        magtype = 'ml'
    else:
        mag = event.preferred_magnitude().mag
        magtype = event.preferred_magnitude().magnitude_type.lower()
    if event.preferred_origin().extra.catalog.value is None:
        auth = 'unknown'
    else:
        auth = event.preferred_origin().extra.catalog.value.replace(' ','')
        
    event_out = event_out.format(
        evid=evid,
        origin=str(origin_time),
        jdate='%s%s'%(origin_time.year,origin_time.julday),
        lon=evlo,
        lat=evla,
        depth=depth/1E3,
        mag=mag,
        auth=auth
        )
        
    outfile = 'quakes.csv'
    with open(outfile,'w') as f:
        f.write('evid,origin,jdate,lon,lat,depth,%s,auth\n'%magtype)
        f.write(event_out)
        
    # dowanload waveforms and metadata
    mseed_storage = '%s/waveforms'%outdir
    stationxml_storage = '%s/stations'%outdir
    mdl = MassDownloader(providers=['NCEDC'])
    mdl_helper = mdl.download(domain, restrictions,
                              mseed_storage=mseed_storage,stationxml_storage=stationxml_storage)

print('%s download completed.'%outdir)

[2023-04-04 01:50:14,414] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for NCEDC.
[2023-04-04 01:50:14,414] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for NCEDC.
[2023-04-04 01:50:14,414] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for NCEDC.
[2023-04-04 01:50:14,419] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): NCEDC.
[2023-04-04 01:50:14,419] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): NCEDC.
[2023-04-04 01:50:14,419] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): NCEDC.
[2023-04-04 01:50:14,421] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2023-04-04 01:50:14,421] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2023-04-04 01:50:14,421] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexis

40191336 download completed.


Now we've downloaded the raw waveforms, the next step is to processing them.